# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [7]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [8]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,state,shortname
0,Dade,12345,Rick Scott,Florida,FL
1,Broward,40000,Rick Scott,Florida,FL
2,Palm Beach,60000,Rick Scott,Florida,FL
3,Summit,1234,John Kasich,Ohio,OH
4,Cuyahoga,1337,John Kasich,Ohio,OH


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [165]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

[{u'_id': {u'$oid': u'52b213b38594d8a2be17c780'},
  u'approvalfy': 1999,
  u'board_approval_month': u'November',
  u'boardapprovaldate': u'2013-11-12T00:00:00Z',
  u'borrower': u'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  u'closingdate': u'2018-07-07T00:00:00Z',
  u'country_namecode': u'Federal Democratic Republic of Ethiopia!$!ET',
  u'countrycode': u'ET',
  u'countryname': u'Federal Democratic Republic of Ethiopia',
  u'countryshortname': u'Ethiopia',
  u'docty': u'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  u'envassesmentcategorycode': u'C',
  u'grantamt': 0,
  u'ibrdcommamt': 0,
  u'id': u'P129828',
  u'idacommamt': 130000000,
  u'impagency': u'MINISTRY OF EDUCATION',
  u'lendinginstr': u'Investment Project Financing',
  u'lendinginstrtype': u'IN',
  u'lendprojectcost': 550000000,
  u'majorsector_percent': [{u'Name': u'Education', u'Percent': 46},
   {u'Name': u'Education', u'Percent': 26},
   {u'Name': u'Public Administration, Law, and Ju

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

### Let's find the top 10 countries with most projects:

In [243]:
pd.read_json('data/world_bank_projects.json')['countryname'].value_counts()[:10]

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Nepal                              12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

#### Now let's get the top 10 major project themes (and for fun see where these projects went)

In [54]:
sample_json = json.load((open('data/world_bank_projects.json')))

In [244]:
top10_projects = json_normalize(sample_json, 'mjtheme_namecode', ['countryname'])
top10_projects.head(20)

,code,name,countryname
0,8,Human development,Federal Democratic Republic of Ethiopia
1,11,,Federal Democratic Republic of Ethiopia
2,1,Economic management,Republic of Tunisia
3,6,Social protection and risk management,Republic of Tunisia
4,5,Trade and integration,Tuvalu
5,2,Public sector governance,Tuvalu
6,11,Environment and natural resources management,Tuvalu
7,6,Social protection and risk management,Tuvalu
8,7,Social dev/gender/inclusion,Republic of Yemen
9,7,Social dev/gender/inclusion,Republic of Yemen


We see some entries only have codes but the names are missing. Further inspection suggests that the codes associated with the names are in the dataframe, but missed during data entry. I replaced the data entries with missing values using nan from numpy, get the codes associated with names in a separate dataframe and map it to the original dataframe. 

In [234]:
tt= top10_projects
tt = tt.replace('', np.nan)
tt1= tt[['code', 'name']]
tt2=tt1.dropna().groupby('code').last()

In [247]:
tt.loc[tt['name'].isnull(),'name'] = tt['code'].map(tt2.name)
tt= tt.set_index('code')
tt.head(20)

,name,countryname
code,,
8,Human development,Federal Democratic Republic of Ethiopia
11,Environment and natural resources management,Federal Democratic Republic of Ethiopia
1,Economic management,Republic of Tunisia
6,Social protection and risk management,Republic of Tunisia
5,Trade and integration,Tuvalu
2,Public sector governance,Tuvalu
11,Environment and natural resources management,Tuvalu
6,Social protection and risk management,Tuvalu
7,Social dev/gender/inclusion,Republic of Yemen


And now we are set to get the major project themes and respective countries.

In [249]:
tt.name.value_counts()

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Rule of law                                      15
Name: name, dtype: int64

In [242]:
tt.countryname.value_counts()[:10]

Republic of Indonesia              56
Republic of India                  51
Socialist Republic of Vietnam      43
Federative Republic of Brazil      41
People's Republic of Bangladesh    41
People's Republic of China         40
Africa                             39
Republic of Yemen                  34
Kingdom of Morocco                 32
Republic of Mozambique             31
Name: countryname, dtype: int64

Hmmm, now I see Indonesia has the most major themed projects, must be very diverse! And Africa is counted as a country? Again Yemen, Morocco and Mozambique are counted as separate countries. 

### Found a cool trick to flatten a json file

In [156]:
data2 = json.load((open('data/world_bank_projects_less.json')))

In [155]:
from flatten_json import flatten_json

In [157]:
flattened = [flatten_json(element) for element in data2]  # # flatten each element in dictionary
flattened

[{'_id_$oid': '52b213b38594d8a2be17c780',
  'approvalfy': '1999',
  'board_approval_month': 'November',
  'boardapprovaldate': '2013-11-12T00:00:00Z',
  'borrower': 'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  'closingdate': '2018-07-07T00:00:00Z',
  'country_namecode': 'Federal Democratic Republic of Ethiopia!$!ET',
  'countrycode': 'ET',
  'countryname': 'Federal Democratic Republic of Ethiopia',
  'countryshortname': 'Ethiopia',
  'docty': 'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  'envassesmentcategorycode': 'C',
  'grantamt': '0',
  'ibrdcommamt': '0',
  'id': 'P129828',
  'idacommamt': '130000000',
  'impagency': 'MINISTRY OF EDUCATION',
  'lendinginstr': 'Investment Project Financing',
  'lendinginstrtype': 'IN',
  'lendprojectcost': '550000000',
  'majorsector_percent_0_Name': 'Education',
  'majorsector_percent_0_Percent': '46',
  'majorsector_percent_1_Name': 'Education',
  'majorsector_percent_1_Percent': '26',
  'majorsector_perce

In [252]:
df = pd.DataFrame(flattened)
df.shape

(2, 115)

**Notice** now df has 115 columns compared to 55 earlier.